# Transfer Learning with RAG Models
----
**Objective**: In this notebook, you will experiment with a QA RAG model on the climate_fever dataset using the Haystack framework. You will get to go through the whole process of fitting the parts of a RAG model together, and learn how to prompt it with queries to get answers from the provided dataset.

NOTE: Make sure to change the runtime from CPU to TPU or GPU for faster training

## Install Libraries
Install the Haystack (for colab) and Datasets libraries

In [1]:
!pip install farm-haystack[colab]
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 768.2/768.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


## Import Dataset
----
In this section, we will use as an example the climate_fever dataset. The dataset consists of 1535 rows of claims about climate change, and they either refute or support climate change, with some claims being neutral. We will build with a specific topic in mind so that we can get more accurate answers, and keep in mind that bigger datasets with open topics can also be used.

**Question 1**: Use the "load_dataset" function to load the "climate_fever" dataset with the "test" split.

In [2]:
from datasets import load_dataset

# Load the dataset
data = load_dataset("climate_fever", split='test')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating test split:   0%|          | 0/1535 [00:00<?, ? examples/s]

In [4]:
data

Dataset({
    features: ['claim_id', 'claim', 'claim_label', 'evidences'],
    num_rows: 1535
})

## Formatting and Writing Documents
----
First, we need to extract, format and write the documents from our chosen dataset so that we can later build our QA Pipeline. This Pipeline will facilitate the process of building our RAG model and getting answers from it.

Keep in mind that for this notebook we will focus on how to build the pipeline with the simplest configurations. Feel free to experiment with different parameters.



**Question 2**: Use the write_documents method to save the formatted documents into document_storage

In [3]:

from haystack.document_stores import InMemoryDocumentStore

# Extract and format the documents from the dataset
documents = [{'content': x['claim']} for x in data]


document_storage = InMemoryDocumentStore(use_bm25=True)

# Write the documents to the document store
document_storage.write_documents(documents)


Updating BM25 representation...: 100%|██████████| 1535/1535 [00:00<00:00, 98957.23 docs/s]


## Preparing the Retriever
----
We need to prepare our Retriever node of our pipeline. It will be responsible to get the documents from our document storage, so that they can be used by the Language Model later. We will the BM25Retriever provided by haystack, as it is the recommended Retriever for begginners.

**Question 3**: Create the BM25Retriever using the document_storage created earlier, with a top_k of value 2

In [7]:
from haystack.nodes import BM25Retriever

# Note: The higher the top_k is, the better the answer will be. However, speed will be affected
retriever = BM25Retriever(document_store = document_storage, top_k=2)


In [8]:
retriever

## Preparing the Language Model
----
Now, we will prepare our Language Model using the prompt node. We need to first create our prompt, and for that, Haystack requires a specific structure. We will then define our desired language model alongside the prompt template we created. When creating this template, we need to Parse the output to a format that Haystack can use.

**Question 4**: Define the prompt node using PromptNode with the model name as "google/flan-t5-large" and the default prompt template as the created "rag_prompt"

In [14]:
from haystack.nodes import PromptNode, PromptTemplate, AnswerParser

rag_prompt = PromptTemplate(
    prompt="""Create comprehensive answers from the related text given the questions.
                             Provide a clear and concise response that displays the key points and information presented in the related text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    output_parser=AnswerParser(),
)

prompt_node = PromptNode(model_name_or_path= "google/flan-t5-large", default_prompt_template=rag_prompt)



config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

## Fitting our Pipeline Together
----
Finally, we are going to put our pipeline nodes together. For that we will use the Pipeline function from haystack. With the pipeline ready you will be able to ask it questions and get answers

**Question 5**: Add the retriever node and prompt_node created in the previous steps to the Pipeline using the add_node function. Hint: you need to provide the inputs to each of these nodes.

In [15]:
from haystack.pipelines import Pipeline

pipe = Pipeline()
pipe.add_node(component=retriever, name='retriever', inputs=['Query'])
pipe.add_node(component=prompt_node, name='prompt_node', inputs=['retriever'])

## Asking the RAG Model Questions
----
We use the pipeline .run() method to ask a question. Since the output provided by our Prompt Node is a Haystack object, we retrieve in the way provided inside the print() function.

In [16]:
output = pipe.run(query="When did global warming start")

print(output["answers"][0].answer)

The 20th century global warming did not start until 1910.


In [17]:
# Here are some other examples you can use
examples = [
    "Who is most responsible for pollution",
    "What is the biggest damaging factor for the climate?",
    "What are some clean energy sources?",
    "How much does the average temperature of our planet rise per decade?"
]

In [18]:
output = pipe.run(query="Who is most responsible for pollution")

print(output["answers"][0].answer)

The IPCC 95% confidence that humans are responsible for most of the current global warming is simply a summary of the peer-reviewed scientific research.


In [19]:
output = pipe.run(query="What is the biggest damaging factor for the climate?")

print(output["answers"][0].answer)

Burping cows are more damaging to the climate than all the cars on this planet.


In [20]:
output = pipe.run(query="What are some clean energy sources?")

print(output["answers"][0].answer)

Solar, wind and geothermal.


In [21]:
output = pipe.run(query="How much does the average temperature of our planet rise per decade?")

print(output["answers"][0].answer)

The average temperature of our planet has risen by around 1.62F (0.9C).
